## Image Folder를 활용하여 커스텀 데이터 셋 생성 후 모델 학습

#### 이미지 데이터 확인하기

In [127]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [128]:
# 이미지 데이터가 들어있는 폴더 살펴보기
!tree

폴더 PATH의 목록입니다.
볼륨 일련 번호는 5375-3C31입니다.
C:.
├─.ipynb_checkpoints
├─MNIST_data
│  └─MNIST
│      └─raw
└─MyData
    ├─test_data
    │  ├─0
    │  └─1
    └─train_data
        ├─0
        └─1


#### 데이터셋 만들기

In [129]:
# 크기 조정
trans = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
                      ])

# 학습용 데이터 생성(59개)
train_data = torchvision.datasets.ImageFolder(root='./MyData/train_data', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 4, shuffle = True, num_workers=2)

# 평가용 데이터 생성(10개)
test_data = torchvision.datasets.ImageFolder(root='./MyData/test_data', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [130]:
# 생성된 데이터 확인
print("학습 전체 데이터 셋 : ",len(data_loader.dataset))
print("학습 배치 묶음 개수 : ", len(data_loader))

print("\n평가 전체 데이터 셋 : ",len(test_set.dataset))
print("평가 배치 묶음 개수 : ", len(test_set))

학습 전체 데이터 셋 :  59
학습 배치 묶음 개수 :  15

평가 전체 데이터 셋 :  10
평가 배치 묶음 개수 :  1


In [131]:
for img,label in data_loader:
    print(img.size())
    print(label)
    break

torch.Size([4, 3, 32, 32])
tensor([0, 0, 1, 0])


#### 모델 생성 및 학습

In [132]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [133]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,4),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,2,4),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(2*5*5, 32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        #print("after layer1 : ",out.shape)
        out = self.layer2(out)
        #print("after layer2 : ",out.shape)
        out = out.view(out.shape[0], -1)
        #print("after view : ",out.shape)
        out = self.layer3(out)
        #print("after layer3 : ",out.shape)
        return out

In [134]:
# 모델 테스트
from torchsummary import summary
net = CNN().to(device)
summary(net,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 29, 29]             294
              ReLU-2            [-1, 6, 29, 29]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4            [-1, 2, 11, 11]             194
              ReLU-5            [-1, 2, 11, 11]               0
         MaxPool2d-6              [-1, 2, 5, 5]               0
            Linear-7                   [-1, 32]           1,632
              ReLU-8                   [-1, 32]               0
            Linear-9                    [-1, 2]              66
Total params: 2,186
Trainable params: 2,186
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.09
Params size (MB): 0.01
Estimated Total Size (MB): 0.11
-----------------------------------------------

In [143]:
net = CNN().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [144]:
print(net)

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 2, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=50, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [145]:
total_batch = len(data_loader)

epochs = 10
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        out = net(imgs)
      
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('(epoch %d) cost = %.5f'%(epoch+1, avg_cost))
print('Learning Finished!')

(epoch 1) cost = 0.69047
(epoch 2) cost = 0.68657
(epoch 3) cost = 0.68131
(epoch 4) cost = 0.67542
(epoch 5) cost = 0.65748
(epoch 6) cost = 0.63865
(epoch 7) cost = 0.60129
(epoch 8) cost = 0.54549
(epoch 9) cost = 0.47623
(epoch 10) cost = 0.38359
Learning Finished!


#### 모델 저장


In [149]:
torch.save(net.state_dict(), "./model/customModel.pth")

#### 저장된 모델 불러오기

In [150]:
new_net = CNN().to(device)
new_net.load_state_dict(torch.load('./model/customModel.pth'))

<All keys matched successfully>

#### 모델 성능 평가

In [151]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = new_net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.800000011920929
